In [ ]:
import xarray as xr
import ultraplot as uplt
import numpy as np

# 1. Cargar los datos REALES que generamos
try:
    # Cargamos los dos modelos para comparar
    ds_cnn = xr.open_dataset('../outputs/hindcast_cnn.nc')
    ds_mlp = xr.open_dataset('../outputs/hindcast_mlp.nc')
    print("✅ Datos cargados: CNN y MLP")
except FileNotFoundError:
    print("❌ ERROR: No se encuentran los archivos en ../outputs/")

# 2. Configuración Global (CORREGIDO: Usamos uplt.rc.update o lo ponemos en axs.format)
# La forma correcta de actualizar settings globales es esta:
uplt.rc.update(linewidth=0.8, ticklabelsize=9, titleloc='l')

# Seleccionar momento
t_idx = 0 
time_str = str(ds_cnn.time.values[t_idx])[:10]

# 3. Crear Figura
fig, axs = uplt.subplots(ncols=3, proj='robin', width='12in')
axs.format(
    coast=True, latlines=20, lonlines=30,
    suptitle=f'Comparación de Modelos: Vapor de Agua (PRW) - {time_str}'
)

# --- PANEL 1: MLP (Red simple) ---
# Usamos la variable 'prw' que es la que existe en tus archivos
m1 = axs[0].contourf(ds_mlp['prw'].isel(time=t_idx), cmap='YlGnBu', levels=20)
axs[0].set_title('(a) MLP (Red Densa)', weight='bold')
axs[0].colorbar(m1, loc='b', label='kg/m²')

# --- PANEL 2: CNN (Tu modelo estrella) ---
m2 = axs[1].contourf(ds_cnn['prw'].isel(time=t_idx), cmap='YlGnBu', levels=20)
axs[1].set_title('(b) CNN Prediction', weight='bold')
axs[1].colorbar(m2, loc='b', label='kg/m²')

# --- PANEL 3: Diferencia (CNN - MLP) ---
# Calculamos la diferencia al vuelo
diff = ds_cnn['prw'].isel(time=t_idx) - ds_mlp['prw'].isel(time=t_idx)

# Escala divergente para ver diferencias positivas/negativas
m3 = axs[2].contourf(diff, cmap='ColdHot', levels=np.linspace(-5, 5, 21), extend='both')
axs[2].set_title('(c) Divergencia (CNN - MLP)', weight='bold')
axs[2].colorbar(m3, loc='b', label='Diferencia (kg/m²)')

# Guardar
# fig.save('../outputs/figures/hindcast_comparison.png', dpi=300) 
# Descomenta la linea de arriba si quieres guardar la imagen

✅ Datos cargados: CNN y MLP


/tmp/ipykernel_3401206/3881777670.py:16: UltraPlotWarning: The rc setting 'linewidth' was deprecated in version 0.8.0 and may be removed in the next major release (version 2.0.0). Please use 'meta.width' instead.
  uplt.rc.update(linewidth=0.8, ticklabelsize=9, titleloc='l')
/tmp/ipykernel_3401206/3881777670.py:23: UltraPlotWarning: GeoAxes can only be shared for rectilinear projections, ax.projection=<Projected CRS: +proj=robin +a=6378137.0 +lon_0=0 +no_defs +type=c ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Robinson
Datum: unknown
- Ellipsoid: unknown
- Prime Meridian: Greenwich
 is not a rectilinear projection.
  fig, axs = uplt.subplots(ncols=3, proj='robin', width='12in')


In [ ]:
import xarray as xr
import ultraplot as uplt
import numpy as np
import matplotlib.pyplot as plt

# Cargar archivos
try:
    cnn = xr.open_dataset('../outputs/hindcast_cnn.nc')['prw']
    mlp = xr.open_dataset('../outputs/hindcast_mlp.nc')['prw']
except:
    print("❌ Faltan archivos hindcast")

uplt.rc.update(linewidth=0.8, ticklabelsize=10, titleloc='c')
t_idx = 6
time_str = str(cnn.time.values[t_idx])[:7]

# --- TRUCO PRO: Definir proyecciones por separado ---
# Los 3 primeros son mapas (Robin), el 4to es un Histograma (None/Cartesiano)
fig = uplt.figure(refwidth='5in')
axs = fig.subplots(nrows=2, ncols=2, proj={1:'robin', 2:'robin', 3:'robin', 4:None})

axs.format(
    suptitle=f'Validación de Arquitecturas: {time_str} (Hindcast)',
    abc=True, abcloc='ul'
)

# --- VERSIÓN RÁPIDA (Pcolormesh) ---
# Bajamos un poco la resolución visual [::2, ::2] para que vuele
mlp_fast = mlp.isel(time=t_idx)[::2, ::2]
cnn_fast = cnn.isel(time=t_idx)[::2, ::2]
diff_fast = cnn_fast - mlp_fast

# 1. MLP
m1 = axs[0].pcolormesh(mlp_fast, cmap='Spectral_r', vmin=0, vmax=70)
axs[0].set_title('Predicción MLP')

# 2. CNN
m2 = axs[1].pcolormesh(cnn_fast, cmap='Spectral_r', vmin=0, vmax=70)
axs[1].set_title('Predicción CNN')

fig.colorbar(m1, loc='r', label='Vapor de Agua (kg/m²)', rows=1)

# 3. Diferencia
m3 = axs[2].pcolormesh(diff_fast, cmap='Balance', vmin=-15, vmax=15)
axs[2].set_title('Diferencia (CNN - MLP)')

# 4. Histograma (Ahora sí en un eje normal, no mapa)
ax_hist = axs[3]
ax_hist.hist(diff_fast.values.flatten(), bins=50, color='gray7')
ax_hist.format(title='Distribución de Errores', xlabel='Diferencia', ylabel='Frecuencia', grid=True)
ax_hist.axvline(0, color='k', ls='--')

fig.colorbar(m3, loc='b', label='Diferencia (kg/m²)', cols=(1,2))

print("✅ Gráfico generado.")
plt.show()